## Task A

### Data Model

[
    `{'__id': 1234567,
    'air_temperature_celcius': 28,
    'date': '27/12/2017',
    'hotspot_data': [ {'confidence': 78, 
                        'datetime': '2017-12-27T04:16:51',
                        'latitude': -37.966,
                        'longitude': 145.051,
                        'surface_temperature_celcius': 68
                        },
                       {'confidence': 82, 
                        'datetime': '2017-12-27T00:02:15',
                        'latitude': -35.541,
                        'longitude': 143.311,
                        'surface_temperature_celcius': 63
                        },
                       {'confidence': 67, 
                        'datetime': '2017-12-27T00:02:15',
                        'latitude': -35.554,
                        'longitude': 143.307,
                        'surface_temperature_celcius': 53
                        },
                       {'confidence': 86, 
                        'datetime': '2017-12-27T00:02:14',
                        'latitude': -35.543,
                        'longitude': 143.316,
                        'surface_temperature_celcius': 67
                        },
                     ],
    'max_wind_speed': 15.9,
    'precipitation': '0.00I',
    'relative_humidity': 58.3,
    'station': 948702,
    'windspeed_knots': 9.3                   
    `}
]

### Justification 
We used Embedding Documents model for modelling relationship between two collections, hotspot_historic and climate_historic with below reasons. 
- In terms of size : We beleived that the document size of embeidding both collection would not exceed 16MB
- In terms of query : We decided to avoid using look-up aggregation(Join) , which would be necessary if we chose Referencing Model. Therefore we decided to use Embedding model. 

## Task B

### Task B.1

In [1]:
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [2]:
import pandas as pd
import json

In [3]:
#reading from csv files
climate_hist_data = pd.read_csv("climate_historic.csv") 
hotspot_hist_data = pd.read_csv("hotspot_historic.csv")

In [4]:
climate_hist_data.columns

Index(['station', 'date', 'air_temperature_celcius', 'relative_humidity',
       'windspeed_knots', 'max_wind_speed', 'precipitation '],
      dtype='object')

In [5]:
climate_hist_data.columns = ['station', 'date', 'air_temperature_celcius', 'relative_humidity',
       'windspeed_knots', 'max_wind_speed', 'precipitation']

In [6]:
hotspot_hist_data.columns

Index(['latitude', 'longitude', 'datetime', 'confidence', 'date',
       'surface_temperature_celcius'],
      dtype='object')

In [7]:
#converting dataframes into json format
climate_json = json.loads(climate_hist_data.to_json(orient='records'))
hotspot_json = json.loads(hotspot_hist_data.to_json(orient='records'))

In [8]:
big_arr = []
for c in climate_json:
    d1 =c['date']
    my_arr = []
    my_dict = c
    big_arr.append(c)
    for h in hotspot_json:
        d2 = h['date']
        temp = dict(h)
        if d1 == d2: #clecking if the records from fire and climate data belong to the same date
            del temp['date'] #deleting repeated date from the embedded record
            my_arr.append(temp)
            
    if my_arr != []: #if fire data exists add a new key to embed the records of fire data
        my_dict['hotspot_data'] = my_arr
big_arr.append(my_dict)

In [9]:
client = MongoClient()

client.drop_database('FIT5148') #drop if the database already exists

db = client.FIT5148
db.New_Collection.drop() #drop if the collection already exists
coll = db.New_collection


result = coll.insert_many(climate_json) #insert data into collection

In [10]:
def check_collection(collection_name): 
    cursor = collection_name.find({})
    for document in cursor: 
        pprint(document)

In [11]:
check_collection(coll)

{'_id': ObjectId('5ce7d9499343692d03f9f7f7'),
 'air_temperature_celcius': 19,
 'date': '31/12/2016',
 'max_wind_speed': 11.1,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.8,
 'station': 948700,
 'windspeed_knots': 7.9}
{'_id': ObjectId('5ce7d9499343692d03f9f7f8'),
 'air_temperature_celcius': 15,
 'date': '2/01/2017',
 'max_wind_speed': 13.0,
 'precipitation': ' 0.02G',
 'relative_humidity': 50.7,
 'station': 948700,
 'windspeed_knots': 9.2}
{'_id': ObjectId('5ce7d9499343692d03f9f7f9'),
 'air_temperature_celcius': 16,
 'date': '3/01/2017',
 'max_wind_speed': 15.0,
 'precipitation': ' 0.00G',
 'relative_humidity': 53.6,
 'station': 948700,
 'windspeed_knots': 8.1}
{'_id': ObjectId('5ce7d9499343692d03f9f7fa'),
 'air_temperature_celcius': 24,
 'date': '4/01/2017',
 'max_wind_speed': 14.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 61.6,
 'station': 948700,
 'windspeed_knots': 7.7}
{'_id': ObjectId('5ce7d9499343692d03f9f7fb'),
 'air_temperature_celcius': 24,
 'date': '5/01/20

                   'longitude': 141.5413,
                   'surface_temperature_celcius': 79},
                  {'confidence': 94,
                   'datetime': '2017-03-14T04:14:30',
                   'latitude': -34.3555,
                   'longitude': 141.5495,
                   'surface_temperature_celcius': 75}],
 'max_wind_speed': 13.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 58.3,
 'station': 948701,
 'windspeed_knots': 7.1}
{'_id': ObjectId('5ce7d9499343692d03f9f840'),
 'air_temperature_celcius': 28,
 'date': '15/03/2017',
 'hotspot_data': [{'confidence': 76,
                   'datetime': '2017-03-15T05:02:00',
                   'latitude': -37.4108,
                   'longitude': 147.0196,
                   'surface_temperature_celcius': 50},
                  {'confidence': 74,
                   'datetime': '2017-03-15T04:59:10',
                   'latitude': -37.4058,
                   'longitude': 147.0117,
                   'surface_temperature_cel

                  {'confidence': 67,
                   'datetime': '2017-03-26T04:39:10',
                   'latitude': -37.1956,
                   'longitude': 141.0252,
                   'surface_temperature_celcius': 44},
                  {'confidence': 74,
                   'datetime': '2017-03-26T04:39:10',
                   'latitude': -38.0542,
                   'longitude': 142.8655,
                   'surface_temperature_celcius': 48}],
 'max_wind_speed': 26.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 60.6,
 'station': 948701,
 'windspeed_knots': 10.1}
{'_id': ObjectId('5ce7d9499343692d03f9f84c'),
 'air_temperature_celcius': 17,
 'date': '27/03/2017',
 'max_wind_speed': 25.1,
 'precipitation': ' 0.16G',
 'relative_humidity': 53.4,
 'station': 948701,
 'windspeed_knots': 9.4}
{'_id': ObjectId('5ce7d9499343692d03f9f84d'),
 'air_temperature_celcius': 18,
 'date': '28/03/2017',
 'hotspot_data': [{'confidence': 88,
                   'datetime': '2017-03-28T04:38:

                   'longitude': 141.6787,
                   'surface_temperature_celcius': 37},
                  {'confidence': 91,
                   'datetime': '2017-04-03T13:17:00',
                   'latitude': -37.8046,
                   'longitude': 146.0304,
                   'surface_temperature_celcius': 43},
                  {'confidence': 83,
                   'datetime': '2017-04-03T13:15:30',
                   'latitude': -37.9068,
                   'longitude': 145.9047,
                   'surface_temperature_celcius': 40},
                  {'confidence': 100,
                   'datetime': '2017-04-03T13:15:10',
                   'latitude': -36.1156,
                   'longitude': 141.6664,
                   'surface_temperature_celcius': 62},
                  {'confidence': 96,
                   'datetime': '2017-04-03T13:15:10',
                   'latitude': -37.8064,
                   'longitude': 146.0189,
                   'surface_temperature_c

                  {'confidence': 92,
                   'datetime': '2017-04-04T04:33:50',
                   'latitude': -36.5989,
                   'longitude': 141.9771,
                   'surface_temperature_celcius': 81},
                  {'confidence': 81,
                   'datetime': '2017-04-04T04:33:40',
                   'latitude': -37.4485,
                   'longitude': 143.5041,
                   'surface_temperature_celcius': 54},
                  {'confidence': 63,
                   'datetime': '2017-04-04T04:33:30',
                   'latitude': -37.1744,
                   'longitude': 143.9256,
                   'surface_temperature_celcius': 43},
                  {'confidence': 100,
                   'datetime': '2017-04-04T04:33:30',
                   'latitude': -36.104,
                   'longitude': 141.772,
                   'surface_temperature_celcius': 113},
                  {'confidence': 69,
                   'datetime': '2017-04-04T04:3

...

{'_id': ObjectId('5ce7d9499343692d03f9f8ff'),
 'air_temperature_celcius': 20,
 'date': '22/09/2017',
 'hotspot_data': [{'confidence': 71,
                   'datetime': '2017-09-22T04:17:10',
                   'latitude': -36.012,
                   'longitude': 141.341,
                   'surface_temperature_celcius': 45}],
 'max_wind_speed': 15.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 43.6,
 'station': 948702,
 'windspeed_knots': 10.3}
{'_id': ObjectId('5ce7d9499343692d03f9f900'),
 'air_temperature_celcius': 17,
 'date': '23/09/2017',
 'hotspot_data': [{'confidence': 70,
                   'datetime': '2017-09-23T23:50:45',
                   'latitude': -37.497,
                   'longitude': 147.266,
                   'surface_temperature_celcius': 37},
                  {'confidence': 86,
                   'datetime': '2017-09-23T23:50:44',
                   'latitude': -37.57,
                   'longitude': 148.034,
                   'surface_temperature_celci

 'station': 948702,
 'windspeed_knots': 5.8}
{'_id': ObjectId('5ce7d9499343692d03f9f90a'),
 'air_temperature_celcius': 13,
 'date': '3/10/2017',
 'hotspot_data': [{'confidence': 69,
                   'datetime': '2017-10-03T15:01:44',
                   'latitude': -37.461,
                   'longitude': 148.109,
                   'surface_temperature_celcius': 33},
                  {'confidence': 92,
                   'datetime': '2017-10-03T13:23:43',
                   'latitude': -37.459,
                   'longitude': 148.092,
                   'surface_temperature_celcius': 42},
                  {'confidence': 68,
                   'datetime': '2017-10-03T03:58:28',
                   'latitude': -37.236,
                   'longitude': 141.176,
                   'surface_temperature_celcius': 37},
                  {'confidence': 59,
                   'datetime': '2017-10-03T03:58:28',
                   'latitude': -37.227,
                   'longitude': 141.151,
  

                   'longitude': 145.179,
                   'surface_temperature_celcius': 52},
                  {'confidence': 62,
                   'datetime': '2017-11-09T04:16:56',
                   'latitude': -36.758,
                   'longitude': 145.19,
                   'surface_temperature_celcius': 51},
                  {'confidence': 75,
                   'datetime': '2017-11-09T04:16:53',
                   'latitude': -37.213,
                   'longitude': 142.941,
                   'surface_temperature_celcius': 51},
                  {'confidence': 85,
                   'datetime': '2017-11-09T04:16:48',
                   'latitude': -37.479,
                   'longitude': 143.358,
                   'surface_temperature_celcius': 59},
                  {'confidence': 72,
                   'datetime': '2017-11-09T04:16:48',
                   'latitude': -37.467,
                   'longitude': 143.351,
                   'surface_temperature_celcius': 47

                  {'confidence': 98,
                   'datetime': '2017-11-30T04:34:57',
                   'latitude': -37.605,
                   'longitude': 149.302,
                   'surface_temperature_celcius': 83},
                  {'confidence': 81,
                   'datetime': '2017-11-30T04:34:57',
                   'latitude': -37.627,
                   'longitude': 149.261,
                   'surface_temperature_celcius': 57},
                  {'confidence': 84,
                   'datetime': '2017-11-30T04:34:57',
                   'latitude': -37.623,
                   'longitude': 149.284,
                   'surface_temperature_celcius': 61},
                  {'confidence': 99,
                   'datetime': '2017-11-30T04:34:57',
                   'latitude': -37.6,
                   'longitude': 149.325,
                   'surface_temperature_celcius': 86},
                  {'confidence': 69,
                   'datetime': '2017-11-30T04:34:57',
   

 'date': '1/01/2018',
 'max_wind_speed': 15.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 52.9,
 'station': 948702,
 'windspeed_knots': 8.1}


### Task B.2

#### (a) Find climate data on 10th December 2017

In [12]:
results = coll.find({"date":"10/12/2017"},{"_id":0})
for result in results:
    pprint(result)

{'air_temperature_celcius': 17,
 'date': '10/12/2017',
 'hotspot_data': [{'confidence': 50,
                   'datetime': '2017-12-10T00:57:59',
                   'latitude': -37.987,
                   'longitude': 144.005,
                   'surface_temperature_celcius': 38},
                  {'confidence': 67,
                   'datetime': '2017-12-10T00:56:57',
                   'latitude': -34.289,
                   'longitude': 141.712,
                   'surface_temperature_celcius': 54}],
 'max_wind_speed': 14.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 53.5,
 'station': 948702,
 'windspeed_knots': 7.3}


#### (b) Find   the   latitude ​ ,   ​ longitude,   surface   temperature   ​ (°C)​ ,   ​ and   ​ confidence   ​ when  the surface temperature (°C) was between ​ 65 °C ​ and​ 100 °C . ​

In [13]:
result2 = coll.aggregate([
    {"$unwind":"$hotspot_data"},
    {"$match":{"hotspot_data.surface_temperature_celcius":{"$gte": 65,  "$lte": 100}}},
    {"$project":{"hotspot_data.latitude":1,"hotspot_data.longitude":1,"hotspot_data.confidence":1, "hotspot_data.surface_temperature_celcius":1,"_id":0}}
     ])
for i in result2:
    pprint(i)

{'hotspot_data': {'confidence': 94,
                  'latitude': -37.2284,
                  'longitude': 147.9187,
                  'surface_temperature_celcius': 73}}
{'hotspot_data': {'confidence': 97,
                  'latitude': -37.6572,
                  'longitude': 142.0703,
                  'surface_temperature_celcius': 80}}
{'hotspot_data': {'confidence': 84,
                  'latitude': -37.0193,
                  'longitude': 148.1459,
                  'surface_temperature_celcius': 71}}
{'hotspot_data': {'confidence': 100,
                  'latitude': -37.4229,
                  'longitude': 147.027,
                  'surface_temperature_celcius': 99}}
{'hotspot_data': {'confidence': 80,
                  'latitude': -37.0055,
                  'longitude': 148.1582,
                  'surface_temperature_celcius': 68}}
{'hotspot_data': {'confidence': 85,
                  'latitude': -37.4128,
                  'longitude': 147.0242,
                  'surface_t

                  'surface_temperature_celcius': 66}}
{'hotspot_data': {'confidence': 90,
                  'latitude': -36.1615,
                  'longitude': 141.8728,
                  'surface_temperature_celcius': 66}}
{'hotspot_data': {'confidence': 90,
                  'latitude': -36.1122,
                  'longitude': 142.1491,
                  'surface_temperature_celcius': 76}}
{'hotspot_data': {'confidence': 100,
                  'latitude': -37.3735,
                  'longitude': 145.8384,
                  'surface_temperature_celcius': 95}}
{'hotspot_data': {'confidence': 89,
                  'latitude': -35.4255,
                  'longitude': 143.6306,
                  'surface_temperature_celcius': 65}}
{'hotspot_data': {'confidence': 96,
                  'latitude': -36.1955,
                  'longitude': 145.914,
                  'surface_temperature_celcius': 78}}
{'hotspot_data': {'confidence': 85,
                  'latitude': -35.4533,
               

                  'longitude': 145.2196,
                  'surface_temperature_celcius': 71}}
{'hotspot_data': {'confidence': 100,
                  'latitude': -37.0899,
                  'longitude': 141.0238,
                  'surface_temperature_celcius': 90}}
{'hotspot_data': {'confidence': 95,
                  'latitude': -36.3823,
                  'longitude': 141.3146,
                  'surface_temperature_celcius': 77}}
{'hotspot_data': {'confidence': 90,
                  'latitude': -36.1462,
                  'longitude': 145.2096,
                  'surface_temperature_celcius': 66}}
{'hotspot_data': {'confidence': 93,
                  'latitude': -36.5243,
                  'longitude': 142.0839,
                  'surface_temperature_celcius': 72}}
{'hotspot_data': {'confidence': 98,
                  'latitude': -37.618,
                  'longitude': 143.0013,
                  'surface_temperature_celcius': 82}}
{'hotspot_data': {'confidence': 90,
              

{'hotspot_data': {'confidence': 92,
                  'latitude': -36.0749,
                  'longitude': 143.4058,
                  'surface_temperature_celcius': 70}}
{'hotspot_data': {'confidence': 91,
                  'latitude': -36.4449,
                  'longitude': 140.9836,
                  'surface_temperature_celcius': 68}}
{'hotspot_data': {'confidence': 100,
                  'latitude': -36.3972,
                  'longitude': 141.6041,
                  'surface_temperature_celcius': 90}}
{'hotspot_data': {'confidence': 90,
                  'latitude': -36.5532,
                  'longitude': 142.5349,
                  'surface_temperature_celcius': 67}}
{'hotspot_data': {'confidence': 92,
                  'latitude': -36.1367,
                  'longitude': 145.2071,
                  'surface_temperature_celcius': 71}}
{'hotspot_data': {'confidence': 96,
                  'latitude': -36.1518,
                  'longitude': 144.0233,
                  'surface_

{'hotspot_data': {'confidence': 94,
                  'latitude': -37.7236,
                  'longitude': 142.947,
                  'surface_temperature_celcius': 73}}
{'hotspot_data': {'confidence': 100,
                  'latitude': -37.7002,
                  'longitude': 142.9562,
                  'surface_temperature_celcius': 89}}
{'hotspot_data': {'confidence': 92,
                  'latitude': -37.6498,
                  'longitude': 142.5997,
                  'surface_temperature_celcius': 70}}
{'hotspot_data': {'confidence': 100,
                  'latitude': -37.5728,
                  'longitude': 142.6348,
                  'surface_temperature_celcius': 98}}
{'hotspot_data': {'confidence': 100,
                  'latitude': -37.4352,
                  'longitude': 143.1444,
                  'surface_temperature_celcius': 96}}
{'hotspot_data': {'confidence': 92,
                  'latitude': -36.5828,
                  'longitude': 144.5775,
                  'surface

{'hotspot_data': {'confidence': 97,
                  'latitude': -36.6224,
                  'longitude': 143.5092,
                  'surface_temperature_celcius': 80}}
{'hotspot_data': {'confidence': 96,
                  'latitude': -38.0933,
                  'longitude': 143.9824,
                  'surface_temperature_celcius': 78}}
{'hotspot_data': {'confidence': 91,
                  'latitude': -35.6138,
                  'longitude': 143.8666,
                  'surface_temperature_celcius': 68}}
{'hotspot_data': {'confidence': 95,
                  'latitude': -36.0885,
                  'longitude': 145.0359,
                  'surface_temperature_celcius': 76}}
{'hotspot_data': {'confidence': 96,
                  'latitude': -36.3193,
                  'longitude': 145.9162,
                  'surface_temperature_celcius': 78}}
{'hotspot_data': {'confidence': 98,
                  'latitude': -36.0652,
                  'longitude': 146.0764,
                  'surface_t

                  'surface_temperature_celcius': 65}}
{'hotspot_data': {'confidence': 96,
                  'latitude': -38.127,
                  'longitude': 143.82,
                  'surface_temperature_celcius': 77}}
{'hotspot_data': {'confidence': 100,
                  'latitude': -37.294,
                  'longitude': 141.232,
                  'surface_temperature_celcius': 87}}
{'hotspot_data': {'confidence': 99,
                  'latitude': -37.293,
                  'longitude': 141.245,
                  'surface_temperature_celcius': 85}}
{'hotspot_data': {'confidence': 91,
                  'latitude': -37.247,
                  'longitude': 141.278,
                  'surface_temperature_celcius': 68}}
{'hotspot_data': {'confidence': 92,
                  'latitude': -36.098,
                  'longitude': 143.74,
                  'surface_temperature_celcius': 67}}
{'hotspot_data': {'confidence': 90,
                  'latitude': -37.331,
                  'longitud

#### (c) Find   ​ date , ​   ​ surface   temperature   (°C),   air   temperature   (°C),   relative   humidity   and  max wind speed on ​ 15th and 16th of December 2017.  

In [14]:
result3 = coll.find({"$or":[{"date":"15/12/2017"},{"date":"16/12/2017"}]},
                    {"_id":0, "date":1, "hotspot_data.surface_temperature_celcius":1,
                    "air_temperature_celcius":1, "relative_humidity":1,"max_wind_speed":1  })
for i in result3:
    pprint(i)

{'air_temperature_celcius': 18,
 'date': '15/12/2017',
 'hotspot_data': [{'surface_temperature_celcius': 42},
                  {'surface_temperature_celcius': 36},
                  {'surface_temperature_celcius': 38},
                  {'surface_temperature_celcius': 40}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18,
 'date': '16/12/2017',
 'hotspot_data': [{'surface_temperature_celcius': 43},
                  {'surface_temperature_celcius': 33},
                  {'surface_temperature_celcius': 54},
                  {'surface_temperature_celcius': 73},
                  {'surface_temperature_celcius': 55},
                  {'surface_temperature_celcius': 75},
                  {'surface_temperature_celcius': 55},
                  {'surface_temperature_celcius': 66},
                  {'surface_temperature_celcius': 56},
                  {'surface_temperature_celcius': 60},
                  {'surface_temperature_celcius': 73},
           

#### (d) Find  datetime,   air   temperature   (°C),   surface   temperature   (°C)   and    confidence  when the  confidence is between 80 and 100.  

In [15]:
result4 = coll.aggregate([
    {"$unwind":"$hotspot_data"},
    {"$match":{"hotspot_data.confidence":{"$gt": 80,  "$lt": 100}}},
    {"$project":{"hotspot_data.datetime":1,"air_temperature_celcius":1,"hotspot_data.confidence":1, "hotspot_data.surface_temperature_celcius":1,"_id":0}}
     ])
for i in result4:
    pprint(i)

{'air_temperature_celcius': 20,
 'hotspot_data': {'confidence': 87,
                  'datetime': '2017-03-06T05:06:30',
                  'surface_temperature_celcius': 62}}
{'air_temperature_celcius': 20,
 'hotspot_data': {'confidence': 85,
                  'datetime': '2017-03-06T05:06:20',
                  'surface_temperature_celcius': 59}}
{'air_temperature_celcius': 19,
 'hotspot_data': {'confidence': 88,
                  'datetime': '2017-03-07T04:16:10',
                  'surface_temperature_celcius': 64}}
{'air_temperature_celcius': 23,
 'hotspot_data': {'confidence': 86,
                  'datetime': '2017-03-09T13:23:40',
                  'surface_temperature_celcius': 41}}
{'air_temperature_celcius': 19,
 'hotspot_data': {'confidence': 94,
                  'datetime': '2017-03-10T04:45:30',
                  'surface_temperature_celcius': 73}}
{'air_temperature_celcius': 19,
 'hotspot_data': {'confidence': 97,
                  'datetime': '2017-03-10T04:45:30',
    

                  'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 19,
 'hotspot_data': {'confidence': 90,
                  'datetime': '2017-04-06T04:27:00',
                  'surface_temperature_celcius': 76}}
{'air_temperature_celcius': 19,
 'hotspot_data': {'confidence': 83,
                  'datetime': '2017-04-06T04:27:00',
                  'surface_temperature_celcius': 57}}
{'air_temperature_celcius': 19,
 'hotspot_data': {'confidence': 83,
                  'datetime': '2017-04-06T04:24:40',
                  'surface_temperature_celcius': 64}}
{'air_temperature_celcius': 19,
 'hotspot_data': {'confidence': 89,
                  'datetime': '2017-04-06T04:24:20',
                  'surface_temperature_celcius': 65}}
{'air_temperature_celcius': 19,
 'hotspot_data': {'confidence': 89,
                  'datetime': '2017-04-06T04:24:20',
                  'surface_temperature_celcius': 65}}
{'air_temperature_celcius': 19,
 'hotspot_data': {'confidence': 89,
   

                  'surface_temperature_celcius': 61}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 81,
                  'datetime': '2017-04-13T04:29:50',
                  'surface_temperature_celcius': 55}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 81,
                  'datetime': '2017-04-13T04:29:40',
                  'surface_temperature_celcius': 54}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 88,
                  'datetime': '2017-04-13T04:29:40',
                  'surface_temperature_celcius': 64}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 89,
                  'datetime': '2017-04-13T04:28:50',
                  'surface_temperature_celcius': 65}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 84,
                  'datetime': '2017-04-13T04:28:30',
                  'surface_temperature_celcius': 58}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 90,
   

                  'surface_temperature_celcius': 64}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 91,
                  'datetime': '2017-04-13T04:26:30',
                  'surface_temperature_celcius': 68}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 81,
                  'datetime': '2017-04-13T04:26:30',
                  'surface_temperature_celcius': 54}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 81,
                  'datetime': '2017-04-13T04:26:30',
                  'surface_temperature_celcius': 54}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 81,
                  'datetime': '2017-04-13T04:26:30',
                  'surface_temperature_celcius': 55}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 82,
                  'datetime': '2017-04-13T04:26:30',
                  'surface_temperature_celcius': 55}}
{'air_temperature_celcius': 16,
 'hotspot_data': {'confidence': 90,
   

                  'surface_temperature_celcius': 72}}
{'air_temperature_celcius': 15,
 'hotspot_data': {'confidence': 94,
                  'datetime': '2017-04-18T04:44:50',
                  'surface_temperature_celcius': 75}}
{'air_temperature_celcius': 15,
 'hotspot_data': {'confidence': 84,
                  'datetime': '2017-04-18T04:44:50',
                  'surface_temperature_celcius': 58}}
{'air_temperature_celcius': 15,
 'hotspot_data': {'confidence': 89,
                  'datetime': '2017-04-18T04:44:50',
                  'surface_temperature_celcius': 65}}
{'air_temperature_celcius': 15,
 'hotspot_data': {'confidence': 91,
                  'datetime': '2017-04-18T00:30:50',
                  'surface_temperature_celcius': 69}}
{'air_temperature_celcius': 15,
 'hotspot_data': {'confidence': 89,
                  'datetime': '2017-04-18T00:30:40',
                  'surface_temperature_celcius': 66}}
{'air_temperature_celcius': 15,
 'hotspot_data': {'confidence': 93,
   

 'hotspot_data': {'confidence': 96,
                  'datetime': '2017-05-05T03:50:20',
                  'surface_temperature_celcius': 78}}
{'air_temperature_celcius': 12,
 'hotspot_data': {'confidence': 91,
                  'datetime': '2017-05-06T04:39:00',
                  'surface_temperature_celcius': 68}}
{'air_temperature_celcius': 12,
 'hotspot_data': {'confidence': 95,
                  'datetime': '2017-05-06T04:33:00',
                  'surface_temperature_celcius': 76}}
{'air_temperature_celcius': 12,
 'hotspot_data': {'confidence': 96,
                  'datetime': '2017-05-06T04:32:50',
                  'surface_temperature_celcius': 78}}
{'air_temperature_celcius': 12,
 'hotspot_data': {'confidence': 85,
                  'datetime': '2017-05-06T04:32:30',
                  'surface_temperature_celcius': 59}}
{'air_temperature_celcius': 12,
 'hotspot_data': {'confidence': 98,
                  'datetime': '2017-05-06T04:32:20',
                  'surface_temperatu

                  'surface_temperature_celcius': 38}}
{'air_temperature_celcius': 14,
 'hotspot_data': {'confidence': 84,
                  'datetime': '2017-09-24T13:30:07',
                  'surface_temperature_celcius': 50}}
{'air_temperature_celcius': 14,
 'hotspot_data': {'confidence': 86,
                  'datetime': '2017-09-24T04:04:18',
                  'surface_temperature_celcius': 76}}
{'air_temperature_celcius': 14,
 'hotspot_data': {'confidence': 84,
                  'datetime': '2017-09-24T04:04:04',
                  'surface_temperature_celcius': 52}}
{'air_temperature_celcius': 15,
 'hotspot_data': {'confidence': 83,
                  'datetime': '2017-09-27T04:35:36',
                  'surface_temperature_celcius': 56}}
{'air_temperature_celcius': 15,
 'hotspot_data': {'confidence': 86,
                  'datetime': '2017-09-27T04:35:36',
                  'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 14,
 'hotspot_data': {'confidence': 82,
   

#### (e) Find the top 10 records with the highest ​ surface temperature ​ ​ (°C). 

In [16]:
result5 = coll.aggregate([
    {"$unwind":"$hotspot_data"},
    {"$sort":{"hotspot_data.surface_temperature_celcius" : -1}},
    {"$limit":10},
    {"$project":{"_id":0}}
    ])
for i in result5:
    pprint(i)

{'air_temperature_celcius': 15,
 'date': '18/04/2017',
 'hotspot_data': {'confidence': 100,
                  'datetime': '2017-04-18T04:52:00',
                  'latitude': -38.1665,
                  'longitude': 143.062,
                  'surface_temperature_celcius': 124},
 'max_wind_speed': 9.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'air_temperature_celcius': 16,
 'date': '4/04/2017',
 'hotspot_data': {'confidence': 100,
                  'datetime': '2017-04-04T04:32:50',
                  'latitude': -36.343,
                  'longitude': 142.1986,
                  'surface_temperature_celcius': 123},
 'max_wind_speed': 12.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 47.5,
 'station': 948701,
 'windspeed_knots': 5.4}
{'air_temperature_celcius': 14,
 'date': '1/05/2017',
 'hotspot_data': {'confidence': 100,
                  'datetime': '2017-05-01T04:14:20',
                  'latitude': -36.9318,
      

#### (f) Find   the   number   of   fire   in   each   day.   You   are   required   to   only   display   ​ the  total number of fire ​ and ​ the date ​ in the output. 

In [17]:
result6 = coll.aggregate([
    {"$unwind":"$hotspot_data"},
    {"$group":{"_id":"$date", "number_of_fire":{"$sum":1}}}
    ])
for i in result6:
    pprint(i)

{'_id': '27/12/2017', 'number_of_fire': 4}
{'_id': '25/12/2017', 'number_of_fire': 1}
{'_id': '15/12/2017', 'number_of_fire': 4}
{'_id': '14/12/2017', 'number_of_fire': 1}
{'_id': '30/11/2017', 'number_of_fire': 31}
{'_id': '14/11/2017', 'number_of_fire': 3}
{'_id': '12/11/2017', 'number_of_fire': 5}
{'_id': '11/11/2017', 'number_of_fire': 4}
{'_id': '22/11/2017', 'number_of_fire': 2}
{'_id': '9/11/2017', 'number_of_fire': 10}
{'_id': '8/11/2017', 'number_of_fire': 2}
{'_id': '5/11/2017', 'number_of_fire': 4}
{'_id': '10/12/2017', 'number_of_fire': 2}
{'_id': '27/10/2017', 'number_of_fire': 5}
{'_id': '23/10/2017', 'number_of_fire': 1}
{'_id': '16/10/2017', 'number_of_fire': 1}
{'_id': '15/10/2017', 'number_of_fire': 3}
{'_id': '10/10/2017', 'number_of_fire': 3}
{'_id': '9/10/2017', 'number_of_fire': 1}
{'_id': '16/12/2017', 'number_of_fire': 15}
{'_id': '18/10/2017', 'number_of_fire': 6}
{'_id': '7/10/2017', 'number_of_fire': 1}
{'_id': '20/10/2017', 'number_of_fire': 3}
{'_id': '6/10

#### (g) Find   the   ​ average   surface   temperature   ​ (°C)   f ​ or   each   day​ .   You   are   required   to  only display ​ average surface temperature (°C) ​ ​ and ​ the date ​ in the output.

In [18]:
result7 = coll.aggregate([
    {"$unwind":"$hotspot_data"},
    {"$group":{"_id":"$date", "average_surface_temp":{"$avg":"$hotspot_data.surface_temperature_celcius"}}}
    ])
for i in result7:
    pprint(i)

{'_id': '27/12/2017', 'average_surface_temp': 62.75}
{'_id': '25/12/2017', 'average_surface_temp': 54.0}
{'_id': '15/12/2017', 'average_surface_temp': 39.0}
{'_id': '14/12/2017', 'average_surface_temp': 70.0}
{'_id': '30/11/2017', 'average_surface_temp': 52.41935483870968}
{'_id': '14/11/2017', 'average_surface_temp': 52.0}
{'_id': '12/11/2017', 'average_surface_temp': 53.0}
{'_id': '11/11/2017', 'average_surface_temp': 46.25}
{'_id': '22/11/2017', 'average_surface_temp': 61.5}
{'_id': '9/11/2017', 'average_surface_temp': 61.3}
{'_id': '8/11/2017', 'average_surface_temp': 45.5}
{'_id': '5/11/2017', 'average_surface_temp': 58.5}
{'_id': '10/12/2017', 'average_surface_temp': 46.0}
{'_id': '27/10/2017', 'average_surface_temp': 50.4}
{'_id': '23/10/2017', 'average_surface_temp': 38.0}
{'_id': '16/10/2017', 'average_surface_temp': 36.0}
{'_id': '15/10/2017', 'average_surface_temp': 72.66666666666667}
{'_id': '10/10/2017', 'average_surface_temp': 53.333333333333336}
{'_id': '9/10/2017', 'ave